In [ ]:
import io
import os
import sys
import json
import umap
import base64
import imageio
import sklearn
import skimage
import numpy as np
import scipy as sp
import pandas as pd
import scanpy as sc
import anndata as ad
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('../..')
%aimport opencell.imaging.embeddings 
from opencell.imaging.embeddings import AnnDataManager, UmapGrid, TargetThumbnailTile

In [ ]:
adm = AnnDataManager(
    filepath='/Users/keith.cheveralls/image-data/december-results-full-median-vq2-target-vectors-adata.h5ad', 
    log_and_scale=False
)

### Generate a non-clumpy UMAP embedding and create the target_labels dataframe

In [ ]:
# create a non-clumpy UMAP (higher n_neighbors and min_dist than 'usual')
adm.run_umap(n_neighbors=30, min_dist=0.3, plot=True)

### Define the grid size and digitize the UMAP coordinates

In [ ]:
grid = UmapGrid(adata=adm.adata, grid_size=40)

In [ ]:
# bin the raw coordinates into the grid
grid.generate_grid_coords()

In [ ]:
empty_internal_bins = grid.mask_internal_empty_bins(grid.grid_coords, plot=True)

In [ ]:
# 'clean up' the grid by moving extra targets to adjacent empty bins
grid_coords_cleaned = grid.clean_up_grid()

In [ ]:
counts_orig = grid.count_targets_per_grid_coord(grid.grid_coords)
counts_cleaned = grid.count_targets_per_grid_coord(grid_coords_cleaned)

plt.figure(figsize=(15, 5))
plt.imshow(np.concatenate((counts_orig, counts_cleaned), axis=1))

### Aside: a better way to fill in the empty grid bins

In [ ]:
# count the total number of targets in the bins neighboring each empty internal bin 
holes_n = empty_internal_bins * skimage.filters.rank.maximum(counts_orig.astype(int), selem=np.ones((3, 3)))

plt.figure(figsize=(15, 5))
plt.imshow(np.concatenate((counts_orig, holes_n), axis=1))

In [ ]:
# find the internal bin whose neighboring bins have the smallest total number of targets

# IDEA: first fill in the isolated empty internal bins in order,
# according to the number of targets in their neighboring bins (from least to most),
# then do the same for the non-isolated empty internal bins
holes_n[holes_n == 0] = holes_n.max()
np.unravel_index(np.argmin(holes_n, axis=None), holes_n.shape)

### Construct the UMAP tile

In [ ]:
# construct the tile coordinates (the grid coordinates for each target in the tile)
tile_coords = grid.construct_tile(grid_coords_cleaned, kind='coords')
len(tile_coords)

In [ ]:
# load the thumbnail images to construct the tiled UMAP image
grid.load_fov_thumbnails(
    filepath='/Users/keith.cheveralls/image-data/2020-12-08-opencell-best-fov-thumbnails.json'
)

In [ ]:
# construct the tiled image itself
umap_tile = grid.construct_tile(grid_coords_cleaned, kind='image', downsample_by=2)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(umap_tile)

### Create a tiled JPG of all targets thumbnails

In [ ]:
tt = TargetThumbnailTile(thumbnail_scale=2, thumbnail_shape='square')

In [ ]:
tt.load_thumbnails_from_cell_line_metadata(
    '/Users/keith.cheveralls/image-data/2020-12-08-opencell-best-fov-thumbnails.json'
)

In [ ]:
tile, positions = tt.construct_tile()

In [ ]:
plt.imshow(tile)

In [ ]:
tt.get_tile_filename()

In [ ]:
imageio.imsave(
    '/Users/keith.cheveralls/projects/opencell-vis/static/images/2020-12-28_all-thumbnails-100px-circle.png',
    thumbnail_tile,
#     quality=75
)